In [1]:
import os
import numpy as np 
import pandas as pd
import time 

import matplotlib.pyplot as plt
import seaborn as sns
from pmdarima import auto_arima
from utils.helper import split_scale_dataset

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
# Load data
combined_df = pd.read_csv("datasets/combined_data.csv", index_col=0, parse_dates=True)

Split and scale datasets.

In [3]:
train, vali, test = split_scale_dataset(combined_df, train_split=0.7, test_split=0.15)

29136 observations in the train dataset.
6240 observations in the validation dataset. 
6240 observations in the test dataset.


In [4]:
# Standard Scaler
train.describe().round(2)

,DE_load_actual_entsoe_transparency,DE_solar_generation_actual,DE_wind_generation_actual,DE_wind_offshore_generation_actual,DE_wind_onshore_generation_actual,GB_UKM_load_actual_entsoe_transparency,GB_UKM_solar_generation_actual,GB_UKM_wind_generation_actual,GB_UKM_wind_offshore_generation_actual,GB_UKM_wind_onshore_generation_actual,ES_load_actual_entsoe_transparency,ES_solar_generation_actual,ES_wind_onshore_generation_actual,FR_load_actual_entsoe_transparency,FR_solar_generation_actual,FR_wind_onshore_generation_actual,IT_load_actual_entsoe_transparency,IT_solar_generation_actual,IT_wind_onshore_generation_actual
count,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00
mean,2.72,0.70,1.28,1.64,1.18,3.06,0.65,1.65,1.57,1.57,2.35,1.22,1.61,1.90,0.80,1.02,2.31,0.99,1.51
std,1.09,1.09,1.00,1.18,1.00,0.71,1.00,1.04,1.07,1.00,0.99,1.45,0.94,0.93,1.12,0.88,1.06,1.38,1.10
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,1.80,0.00,0.50,0.56,0.43,2.51,0.00,0.78,0.66,0.74,1.50,0.05,0.87,1.20,0.00,0.40,1.39,0.00,0.59
50%,2.68,0.02,1.00,1.52,0.89,3.13,0.03,1.47,1.37,1.40,2.39,0.51,1.43,1.77,0.10,0.74,2.23,0.03,1.25
75%,3.69,1.09,1.78,2.57,1.64,3.57,1.03,2.39,2.35,2.28,3.13,2.20,2.17,2.57,1.48,1.36,3.22,1.89,2.21
max,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00


In [4]:
train.describe().round(2)

,DE_load_actual_entsoe_transparency,DE_solar_generation_actual,DE_wind_generation_actual,DE_wind_offshore_generation_actual,DE_wind_onshore_generation_actual,GB_UKM_load_actual_entsoe_transparency,GB_UKM_solar_generation_actual,GB_UKM_wind_generation_actual,GB_UKM_wind_offshore_generation_actual,GB_UKM_wind_onshore_generation_actual,ES_load_actual_entsoe_transparency,ES_solar_generation_actual,ES_wind_onshore_generation_actual,FR_load_actual_entsoe_transparency,FR_solar_generation_actual,FR_wind_onshore_generation_actual,IT_load_actual_entsoe_transparency,IT_solar_generation_actual,IT_wind_onshore_generation_actual
count,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00,29136.00
mean,5.43,1.40,2.55,3.28,2.37,6.12,1.29,3.30,3.15,3.14,4.70,2.44,3.22,3.81,1.61,2.05,4.62,1.97,3.01
std,2.17,2.18,2.01,2.35,1.99,1.42,1.99,2.08,2.15,1.99,1.99,2.90,1.88,1.86,2.24,1.76,2.11,2.75,2.19
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,3.60,0.00,0.99,1.12,0.85,5.02,0.00,1.57,1.31,1.47,3.00,0.11,1.75,2.40,0.00,0.79,2.77,0.00,1.19
50%,5.37,0.03,2.01,3.04,1.78,6.26,0.05,2.95,2.74,2.80,4.79,1.02,2.86,3.54,0.19,1.48,4.46,0.05,2.50
75%,7.37,2.18,3.56,5.14,3.28,7.15,2.06,4.77,4.71,4.55,6.26,4.39,4.35,5.14,2.96,2.72,6.44,3.77,4.42
max,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00,10.00


# 1. Persistence/ naive forecast

Last 24, 96, 168 hours as "predictions".

In [8]:
# Dictionary to rename columns
rename_col_dict = {
    'load_actual_entsoe_transparency': 'load',
    'solar_generation_actual': 'solar',
    'wind_generation_actual': 'wind',
    'wind_onshore_generation_actual': 'wind_onshore',
    'wind_offshore_generation_actual': 'wind_offshore'
}

# List to store metrics for the DataFrame
data = []

# Loop over each prediction length
for pred_len in [24, 96, 168]:
    
    # Combine the last `pred_len` hours of validation data with test data
    forecast_df = pd.concat([vali.iloc[-pred_len:], test], axis=0)

    # Shift the DataFrame by `pred_len` hours to create the persistence forecast
    forecast_df = forecast_df.shift(freq=f'{pred_len}H')

    # Cut off the last hours, because they are longer by 'window_len' than the test data
    forecast_df = forecast_df.iloc[:-pred_len]

    # Loop over each country
    for country in ['DE', 'GB', 'ES', 'FR', 'IT']:

        # Choose columns that belong to the current country
        country_columns = [col for col in test.columns if col.startswith(country)]

        # Loop over each column in the current country
        for col in country_columns:

            # Rename the column based on the country
            if country == 'GB':
                new_col = rename_col_dict.get(col.split('_', 2)[-1], col)
            else:
                new_col = rename_col_dict.get(col.split('_', 1)[-1], col)

            # Calculate metrics
            mae = mean_absolute_error(test[col], forecast_df[col])
            mse = mean_squared_error(test[col], forecast_df[col])
            rmse = np.sqrt(mse)

            # Append metrics to the list
            data.append({
                'Country': country,
                'Pred_len': pred_len,
                'Column': new_col,
                'MAE': mae,
                'MSE': mse,
                'RMSE': rmse
            })

# Convert the list of dictionaries into a DataFrame
df_persistence = pd.DataFrame(data)

# Set MultiIndex
df_persistence.set_index(['Country', 'Pred_len', 'Column'], inplace=True)

df_persistence = df_persistence.sort_index().round(4)
df_persistence

MAE     MSE    RMSE
Country Pred_len Column                               
DE      24       load           0.3996  0.3726  0.6104
                 solar          0.1988  0.1694  0.4116
                 wind           0.7501  1.0413  1.0204
                 wind_offshore  1.1544  2.2971  1.5156
                 wind_onshore   0.7100  0.9798  0.9898
        96       load           0.7422  0.8852  0.9408
                 solar          0.2942  0.3421  0.5849
                 wind           1.0796  1.9393  1.3926
                 wind_offshore  1.4488  3.2892  1.8136
                 wind_onshore   1.0130  1.8232  1.3503
        168      load           0.1938  0.1025  0.3201
                 solar          0.3251  0.3980  0.6309
                 wind           1.0573  1.9687  1.4031
                 wind_offshore  1.3800  3.0111  1.7352
                 wind_onshore   1.0083  1.8787  1.3707
ES      24       load           0.3320  0.2652  0.5150
                 solar          0.2670  0.2341  0.4838
                 wind_onshore   0.7429  0.9073  0.9525
        96       load           0.6039  0.6247  0.7904
                 solar          0.3722  0.4311  0.6565
                 wind_onshore   1.0081  1.6619  1.2892
        168      load           0.2448  0.1102  0.3320
                 solar          0.4177  0.5353  0.7317
                 wind_onshore   1.0721  1.8234  1.3503
FR      24       load           0.2230  0.1307  0.3615
                 solar          0.1928  0.1515  0.3892
                 wind_onshore   0.8856  1.5619  1.2498
        96       load           0.4274  0.3220  0.5675
                 solar          0.3001  0.3316  0.5759
                 wind_onshore   1.2999  3.0023  1.7327
        168      load           0.2320  0.1467  0.3830
                 solar          0.3170  0.3807  0.6170
                 wind_onshore   1.1960  2.6462  1.6267
GB      24       load           0.2680  0.1553  0.3941
                 solar          0.2527  0.2539  0.5039
                 wind           0.9546  1.5789  1.2565
                 wind_offshore  1.2823  2.9091  1.7056
                 wind_onshore   0.8006  1.1091  1.0532
        96       load           0.3955  0.2826  0.5316
                 solar          0.3368  0.4369  0.6610
                 wind           1.3063  2.6709  1.6343
                 wind_offshore  1.6855  4.5836  2.1409
                 wind_onshore   1.0700  1.8264  1.3514
        168      load           0.2798  0.1510  0.3886
                 solar          0.3722  0.5081  0.7128
                 wind           1.2454  2.4731  1.5726
                 wind_offshore  1.6481  4.4433  2.1079
                 wind_onshore   1.0052  1.6586  1.2879
IT      24       load           0.3674  0.3475  0.5895
                 solar          0.1350  0.0805  0.2836
                 wind_onshore   0.7984  1.0938  1.0459
        96       load           0.6836  0.8241  0.9078
                 solar          0.1802  0.1435  0.3788
                 wind_onshore   1.0440  1.8126  1.3463
        168      load           0.2394  0.1305  0.3612
                 solar          0.1933  0.1532  0.3913
                 wind_onshore   1.0201  1.7070  1.3065

In [9]:
# Group by country and window length
df_persistence_country = df_persistence.groupby(['Country', 'Pred_len']).mean()
df_persistence_country.columns = pd.MultiIndex.from_product([['Persistence'], ['MSE', 'MAE', 'RMSE']], names=['Model', 'Metrics'])
df_persistence_country.round(4)

Model            Persistence                
Metrics                  MSE     MAE    RMSE
Country Pred_len                            
DE      24            0.6426  0.9720  0.9096
        96            0.9156  1.6558  1.2164
        168           0.7929  1.4718  1.0920
ES      24            0.4473  0.4689  0.6504
        96            0.6614  0.9059  0.9120
        168           0.5782  0.8230  0.8047
FR      24            0.4338  0.6147  0.6668
        96            0.6758  1.2186  0.9587
        168           0.5817  1.0579  0.8756
GB      24            0.7116  1.2013  0.9827
        96            0.9588  1.9601  1.2638
        168           0.9101  1.8468  1.2140
IT      24            0.4336  0.5073  0.6397
        96            0.6359  0.9267  0.8776
        168           0.4843  0.6636  0.6863

In [ ]:
# Create a folder named "results" if it doesn't exist
folder_name = "results"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

subfolder_name = os.path.join(folder_name, "naive")
if not os.path.exists(subfolder_name):
    os.makedirs(subfolder_name)

# Store dataframes
df_persistence.to_csv('results/naive/metrics_persistence_columns.csv')
df_persistence_country.to_csv('results/naive/metrics_persistence_countries.csv')

# 2. ARIMA model

1. AutoArima from pmdarima package finds optimal parameters for ARIMA model automatically.

2. However, according description, it is better to set seasonal parameter that we determined from the data(m is for seasonality).

I performed small tests on load and solar columns with m=24 and without. Without m I got bad results, with m=24 - good results. As we saw, we have seasonality in these columns (based on the visual inspection in the 1.Data_analysis notebook as well.)  

Wind generation does not react on m parameter and therefore gives the worst performance among all columns. Since it has no short-term seasonality, it is harder to predict these values on hourly data. Of course, wind is supposed to be stronger during coldest periods of the year. But then we have to put to our model factor * 8760 observations. (24 hours * 365 days)

3. Additionally to that I performed ARIMA (or SARIMA) to find optimal input length for training the model.
Putting 576 data points (24 days) and 2160 (90 days, around 3 months) the latest improved average MSE from 0.34 up to 0.3 (based on 36 models). That means that our data has also some monthly seasonality.

4. Putting exogenous variables like 'HourOfDay' and 'DayOfWeek' detariorates performance. Therefore we use exclusively one time serie as the input.

In [ ]:
# https://alkaline-ml.com/pmdarima/tips_and_tricks.html
# maxiter set btw 10-20, default=50 -> good trade off btw speed and robustness

In [ ]:
# Create a folder named "results" if it doesn't exist
folder_name = "results"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

subfolder_name = os.path.join(folder_name, "arima")
if not os.path.exists(subfolder_name):
    os.makedirs(subfolder_name)

The problem is that training time of ARIMA model with parameter selection is long, especially for seasonal ARIMA. 
Unfortuntely, auto_arima has no GPU support.
We have 19 columns and each has 260 test days. Therefore, it is 4940 potential models.

In [10]:
int(test.__len__()/24 * test.columns.__len__())

4940

I performed ARIMA without exogenous variables on 12 days. So, 12 models for each of 19 columns. Only for load and solar columns seasonal ARIMA was performed. From output we can see that seasonal ARIMA on 12 models took between 70 and 115 minutes. 

Non-seasonal ARIMA on 12 models for all wind columns took around 1 min. The total time for 12*19=228 models is 800 minutes that is more than 13 hours.

In [11]:
start = time.time()

# Define parameters for ARIMA model (30 days)
input_len = 720     #(30 days)
n_models = 12        # number of models trained for each column
counter = 0         # to count number of all models


# List to collect metrics and predictions
arima_data = []

# Loop over each country
for country in ['DE', 'GB', 'ES', 'FR', 'IT']:
    # Loop over each prediction length
    for pred_len in [24, 96, 168]:
        # Loop over each column
        for col in [c for c in train.columns if c.startswith(country)]:
            
            # Determine seasonal parameters based on column content
            if any(prefix in col for prefix in ['load', 'solar']):
                seasonal = True
                m = 24
            elif any(prefix in col for prefix in ['wind']):
                seasonal = False
                m = 1
            
            #seasonal = False
            #m = 1
            
            print(f"Processing column: '{col}' with prediction length: {pred_len}")

            # Choose the last `input_len` hours of validation data for training
            tr_data = vali.iloc[-input_len:][col]

            # Extract true values for the column for the next `pred_len` hours
            true_values = test.iloc[:pred_len*n_models][col]

            rows_list = []

            int_start = time.time()

            # Forecast pred_len hours over n_models separately
            for i in range(n_models):
                idx_start = i * pred_len
                idx_end = (i + 1) * pred_len

                # Select the segment to forecast
                ts_data = test.iloc[idx_start:idx_end][col]

                # Fit the ARIMA model
                model = auto_arima(tr_data, stepwise=True, seasonal=seasonal, m=m, maxiter=10)
                print(f"Best ARIMA parameters: {model.order} {model.seasonal_order}")

                # Predict the next `pred_len` hours
                forecasts, confidence = model.predict(n_periods=pred_len, return_conf_int=True)

                # Append the last actual data to the training data
                tr_data = pd.concat([tr_data, ts_data], axis=0)

                # Limit training data to the last `input_len` hours
                tr_data = tr_data[-input_len:]

                # Calculate metrics
                mae = mean_absolute_error(ts_data, forecasts)
                mse = mean_squared_error(ts_data, forecasts)
                rmse = np.sqrt(mse)

                print(f"MAE: {mae:.2f}, MSE: {mse:.2f}, RMSE: {rmse:.2f}")

                # Store metrics and predictions
                rows_list.append({"MAE": mae, "MSE": mse, "RMSE": rmse})
                counter += 1

            int_end = time.time()
            hours_int, rem_int = divmod(int_end - int_start, 3600)
            mins, secs = divmod(rem_int, 60)

            print(f"Time for {counter} models:", "{:0>2}h:{:0>2}m:{:05.2f}s".format(int(hours_int), int(mins), secs))
            print('-'*75)

            # Rename the column based on the country
            if country == 'GB':
                new_col = rename_col_dict.get(col.split('_', 2)[-1], col)
            else:
                new_col = rename_col_dict.get(col.split('_', 1)[-1], col)
                
            # Collect metrics
            arima_data.append((country, pred_len, new_col, 
                               mean_absolute_error(ts_data, forecasts),
                               mean_squared_error(ts_data, forecasts),
                               np.sqrt(mean_squared_error(ts_data, forecasts))))

end = time.time()
hours, rem = divmod(end - start, 3600)
minutes, seconds = divmod(rem, 60)
print("Total time:", "{:0>2}h:{:0>2}m:{:05.2f}s".format(int(hours), int(minutes), seconds))

Processing column: 'DE_load_actual_entsoe_transparency' with prediction length: 24
Best ARIMA parameters: (2, 1, 2) (2, 0, 2, 24)
MAE: 0.21, MSE: 0.07, RMSE: 0.26
Best ARIMA parameters: (0, 1, 3) (1, 0, 2, 24)
MAE: 0.23, MSE: 0.07, RMSE: 0.27
Best ARIMA parameters: (1, 1, 2) (2, 0, 2, 24)
MAE: 0.10, MSE: 0.02, RMSE: 0.12
Best ARIMA parameters: (4, 1, 2) (2, 0, 2, 24)
MAE: 0.14, MSE: 0.03, RMSE: 0.17
Best ARIMA parameters: (0, 1, 4) (2, 0, 2, 24)
MAE: 0.74, MSE: 0.67, RMSE: 0.82
Best ARIMA parameters: (1, 1, 5) (2, 0, 2, 24)
MAE: 0.24, MSE: 0.10, RMSE: 0.32
Best ARIMA parameters: (0, 1, 3) (2, 0, 2, 24)
MAE: 1.00, MSE: 1.25, RMSE: 1.12
Best ARIMA parameters: (2, 1, 0) (2, 0, 2, 24)
MAE: 0.09, MSE: 0.02, RMSE: 0.13
Best ARIMA parameters: (3, 1, 2) (2, 0, 2, 24)
MAE: 0.22, MSE: 0.08, RMSE: 0.28
Best ARIMA parameters: (0, 1, 3) (2, 0, 2, 24)
MAE: 0.22, MSE: 0.06, RMSE: 0.25
Best ARIMA parameters: (0, 1, 3) (2, 0, 2, 24)
MAE: 0.14, MSE: 0.03, RMSE: 0.16
Best ARIMA parameters: (0, 1, 3) (2, 

In [12]:
# Convert collected data into DataFrames
arima_df = pd.DataFrame(arima_data, columns=['Country', 'Pred_len', 'Column', 'MAE', 'MSE', 'RMSE'])
arima_df.set_index(['Country', 'Pred_len', 'Column'], inplace=True)
arima_df.round(4)

MAE     MSE    RMSE
Country Pred_len Column                               
DE      24       load           0.5363  0.3791  0.6157
                 solar          0.0742  0.0170  0.1306
                 wind           0.2360  0.0643  0.2537
                 wind_offshore  0.3971  0.2075  0.4555
                 wind_onshore   0.2252  0.0702  0.2649
        96       load           0.5965  0.4911  0.7008
                 solar          0.2186  0.1518  0.3896
                 wind           1.1271  1.7406  1.3193
                 wind_offshore  0.7289  0.6745  0.8213
                 wind_onshore   1.0800  1.6742  1.2939
        168      load           0.3944  0.2882  0.5368
                 solar          0.4940  0.6983  0.8356
                 wind           0.7846  0.8132  0.9018
                 wind_offshore  1.0113  1.5463  1.2435
                 wind_onshore   0.6025  0.6390  0.7993
GB      24       load           0.2668  0.1202  0.3468
                 solar          0.0916  0.0163  0.1276
                 wind           1.0284  1.3442  1.1594
                 wind_offshore  1.2207  2.0129  1.4188
                 wind_onshore   0.6445  0.5330  0.7300
        96       load           0.3835  0.2081  0.4562
                 solar          0.4234  0.4076  0.6384
                 wind           1.6334  3.3225  1.8228
                 wind_offshore  1.4355  2.2881  1.5126
                 wind_onshore   1.5997  3.2826  1.8118
        168      load           0.3628  0.2484  0.4984
                 solar          0.3294  0.3194  0.5651
                 wind           0.9243  1.1074  1.0523
                 wind_offshore  1.4179  2.4551  1.5669
                 wind_onshore   1.2779  2.5486  1.5964
ES      24       load           0.8180  0.8208  0.9060
                 solar          0.0964  0.0209  0.1445
                 wind_onshore   0.2207  0.0759  0.2755
        96       load           0.4461  0.3934  0.6272
                 solar          0.4725  0.5833  0.7638
                 wind_onshore   1.2990  2.0839  1.4436
        168      load           0.4437  0.3095  0.5563
                 solar          0.4764  0.4634  0.6807
                 wind_onshore   0.9842  1.4249  1.1937
FR      24       load           0.4929  0.2966  0.5446
                 solar          0.0950  0.0174  0.1318
                 wind_onshore   0.1320  0.0353  0.1878
        96       load           0.3175  0.1439  0.3794
                 solar          0.2435  0.1325  0.3641
                 wind_onshore   1.2507  2.1724  1.4739
        168      load           0.9558  1.2032  1.0969
                 solar          0.4549  0.5639  0.7509
                 wind_onshore   1.3433  2.5285  1.5901
IT      24       load           0.8022  0.8427  0.9180
                 solar          0.1830  0.0898  0.2996
                 wind_onshore   0.3435  0.1758  0.4192
        96       load           0.6083  0.6683  0.8175
                 solar          0.2217  0.1229  0.3506
                 wind_onshore   1.1927  1.7864  1.3366
        168      load           0.2890  0.1612  0.4015
                 solar          0.4472  0.4620  0.6797
                 wind_onshore   1.0016  1.6490  1.2841

In [13]:
# By country
arima_df_country = arima_df.groupby(['Country', 'Pred_len']).mean()
arima_df_country.columns = pd.MultiIndex.from_product([['(S)ARIMA'], ['MSE', 'MAE', 'RMSE']], names=['Model', 'Metrics'])
arima_df_country.round(4)

Model            (S)ARIMA                
Metrics               MSE     MAE    RMSE
Country Pred_len                         
DE      24         0.2937  0.1476  0.3441
        96         0.7502  0.9464  0.9050
        168        0.6574  0.7970  0.8634
ES      24         0.3784  0.3059  0.4420
        96         0.7392  1.0202  0.9448
        168        0.6348  0.7326  0.8102
FR      24         0.2400  0.1164  0.2881
        96         0.6039  0.8163  0.7391
        168        0.9180  1.4318  1.1460
GB      24         0.6504  0.8053  0.7565
        96         1.0951  1.9018  1.2484
        168        0.8625  1.3358  1.0558
IT      24         0.4429  0.3694  0.5456
        96         0.6743  0.8592  0.8349
        168        0.5793  0.7574  0.7885

In [ ]:
start = time.time()

# Define input length of training data for ARIMA model (30 days)
input_len = 720

# Define empty DataFrames to store metrics and predictions
metrics_df = pd.DataFrame()
predictions_df = pd.DataFrame()

counter = 0

for pred_len in [24, 96, 168]:

    for col in train.columns:

        # Determine seasonal parameters based on column content
        if any(prefix in col for prefix in ['load', 'solar']):
            seasonal = True
            m = 24
        elif any(prefix in col for prefix in ['wind']):
            seasonal = False
            m = 1

        print(f"Processing column: {col} with window length: {pred_len}")
        
        # Choose the last `input_len` hours of validation data for training
        tr_data = vali.iloc[-input_len:][col]
        
        # Extract true values for the column for the next `pred_len` hours
        true_values = test.iloc[:pred_len*12][col]
        
        pred_list = []
        rows_list = []

        int_start = time.time()

        for i in range(12):
            idx_start = i * pred_len
            idx_end = (i + 1) * pred_len

            # Select the segment to forecast
            ts_data = test.iloc[idx_start:idx_end][col]

            # Fit the ARIMA model
            model = auto_arima(tr_data, stepwise=True, seasonal=seasonal, m=m, maxiter=10)
            print(f"Best ARIMA parameters: {model.order} {model.seasonal_order}")
            
            # Predict the next `window_length` hours
            forecasts, confidence = model.predict(n_periods=pred_len, return_conf_int=True)

            # Append the last actual data to the training data
            tr_data = pd.concat([tr_data, ts_data], axis=0)

            # Limit training data to the last `input_len` hours
            tr_data = tr_data[-input_len:]

            # Calculate metrics
            mae = mean_absolute_error(ts_data, forecasts)
            mse = mean_squared_error(ts_data, forecasts)
            rmse = np.sqrt(mse)

            # Store metrics and predictions
            rows_list.append({"MAE": mae, "MSE": mse, "RMSE": rmse})
            pred_list.append(forecasts)

            counter += 1

        int_end = time.time()
        hours_int, rem_int = divmod(int_end - int_start, 3600)
        mins, secs = divmod(rem_int, 60)
        print(f"Time for {counter} models:", "{:0>2}h:{:0>2}m:{:05.2f}s".format(int(hours_int), int(mins), secs))

        # Process to store metrics into DataFrame
        df_temp = pd.DataFrame(rows_list)
        cols_multiindex = pd.MultiIndex.from_product(
            [[col], [pred_len], ['MAE', 'MSE', 'RMSE']],
            names=['Column name', 'Window Length', 'Metrics']
        )
        df_temp.columns = cols_multiindex
        metrics_df = pd.concat([metrics_df, df_temp], axis=1)

        # Process to store predictions into DataFrame
        df_temp_pred = pd.DataFrame({
            'True': true_values,
            'Predicted': np.array(pred_list).flatten()
        }, index=true_values.index)
        col_multiindex_pred = pd.MultiIndex.from_product(
            [[col], [pred_len], df_temp_pred.columns],
            names=['Column name', 'Window Length', 'Values']
        )
        df_temp_pred.columns = col_multiindex_pred
        predictions_df = pd.concat([predictions_df, df_temp_pred], axis=1)

end = time.time()
hours, rem = divmod(end - start, 3600)
minutes, seconds = divmod(rem, 60)
print("Total time:", "{:0>2}h:{:0>2}m:{:05.2f}s".format(int(hours), int(minutes), seconds))

# Display results
print("Metrics DataFrame:")
print(metrics_df)
print("\nPredictions DataFrame:")
print(predictions_df)


In [ ]:
import time 
start = time.time()

# Define window length for ARIMA model
input_len = 720 # (30 days)

# Define empty DataFrames to store metrics and values
metrics_df = pd.DataFrame()
predictions_df = pd.DataFrame()

counter = 0

for col in train.columns:

    # Check if column contains load/solar or wind
    if any(prefix in col for prefix in ['load', 'solar']):
        seasonal = True
        m = 24
    elif any(prefix in col for prefix in ['wind']):
        seasonal = False
        # No seasonal parameter needed for non-seasonal data
        m = 1  

    print("Processed column:", col)
    # Choose last 30 days and store it outside of the loop to update
    tr_data = vali.iloc[-input_len:][col]
    
    # Extract true values for the column
    true_values = test.iloc[:24*12][col] 

    pred_list = []
    rows_list = []

    int_start = time.time()
    #for i in range(int(test.shape[0]/24)):
    for i in range(12):
    
        idx_start = i*24
        idx_end = (i+1)*24

        # Choose 24 hours to forecast
        ts_data = test.iloc[idx_start:idx_end][col]

        # Model 
        model = auto_arima(tr_data, stepwise=True, seasonal=seasonal, m=m, maxiter=10)
        print(f"Best ARIMA parameters: {model.order} {model.seasonal_order}")
        forecasts, confidence = model.predict(n_periods=24, return_conf_int=True)

        # Append last actual data to train data
        tr_data = pd.concat([tr_data, ts_data], axis=0)

        # Cut train data
        tr_data = tr_data[-input_len:]

        # Calculate metrics and add them to a list of metrics
        mae = mean_absolute_error(ts_data, forecasts)
        mse = mean_squared_error(ts_data, forecasts)
        rmse = np.sqrt(mse)

        row = {"MAE": mae, "MSE": mse, "RMSE": rmse}

        # Append values to store them outside the loop
        rows_list.append(row)
        pred_list.append(forecasts)

        counter+=1
    int_end = time.time()

    hours_int, rem_int = divmod(int_end-int_start, 3600)
    mins, secs = divmod(rem_int, 60)

    print(f"Time intermediate for {counter} models:", "{:0>2}h:{:0>2}m:{:05.2f}s".format(int(hours_int),int(mins),secs))

    # Process to store metrics into df
    df_temp = pd.DataFrame(rows_list)
    cols_multiindex = pd.MultiIndex.from_product(
        [[col], ['MAE', 'MSE', 'RMSE']],
          names=['Column name', 'Metrics'])
    df_temp.columns = cols_multiindex
    metrics_df = pd.concat([metrics_df, df_temp], axis=1)

    # Process to store values into df
    df_temp_pred = pd.DataFrame({'True': true_values, 
                                 'Predicted': np.array(pred_list).flatten()}, 
                                 index=true_values.index)
    col_multiindex_pred = pd.MultiIndex.from_product(
        [[col], df_temp_pred.columns], 
        names=['Column name', 'Values'])
    df_temp_pred.columns = col_multiindex_pred
    predictions_df = pd.concat([predictions_df, df_temp_pred], axis=1)
    
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("Total time:", "{:0>2}h:{:0>2}m:{:05.2f}s".format(int(hours),int(minutes),seconds))

In [ ]:
# Store results
# metrics_df.to_csv('results/arima/metrics_full.csv')
# predictions_df.to_csv('results/arima/predictions_full.csv')

Here are averaged metrics per column.

In [ ]:
# Load results
# Please note that results have multiindex, therefore we have to read them 
# in a right way
# preds_df = pd.read_csv('datasets/predictions_full.csv', header=[0, 1], index_col=0)

metrics_df = pd.read_csv('results/arima/metrics_full.csv', header=[0, 1], index_col=0)
metrics_df.mean(axis=0)

Here are averaged metrics per country.

In [ ]:
# Loop over multiindex dataframe to get the columns that start with country index
# and also loop over the metrics to average

top_5_countries = ['DE', 'GB', 'ES', 'FR', 'IT']
metrics = ['MAE', 'MSE', 'RMSE']

for country in top_5_countries:
    print(f"Country: {country}")
    country_columns = [(col, metric) for col, metric in metrics_df.columns if col.startswith(country)]
    for metric in metrics:
        metric_columns = [(col, metric) for col, m in country_columns if m == metric]
        if metric_columns:
            values_to_average = metrics_df[metric_columns].values.flatten()
            mean_value = values_to_average.mean()
            print(f"Mean {metric}: {mean_value}")
        else:
            print(f"No data for metric {metric} in country {country}")
    print()


In [ ]:
import time 
start = time.time()

# Define empty DataFrames to store metrics and values
metrics_df = pd.DataFrame()
predictions_df = pd.DataFrame()

counter = 0

for col in train.columns:
    """
    # Check if column contains load/solar or wind
    if any(prefix in col for prefix in ['load', 'solar']):
        seasonal = True
        m = 24
    elif any(prefix in col for prefix in ['wind']):
        seasonal = False
        # No seasonal parameter needed for non-seasonal data
        m = 1  
    """
    seasonal = False
    m = 1

    print("Processed column:", col)
    # Choose last 30 days and store it outside of the loop to update
    tr_data = vali.iloc[-2160:][col]
    
    # Extract true values for the column
    true_values = test.iloc[:24*36][col] 

    pred_list = []
    rows_list = []

    int_start = time.time()
    #for i in range(int(test.shape[0]/24)):
    for i in range(36):
    
        idx_start = i*24
        idx_end = (i+1)*24

        # Choose 24 hours to forecast
        ts_data = test.iloc[idx_start:idx_end][col]

        # Model 
        model = auto_arima(tr_data, stepwise=True, seasonal=seasonal, m=m, maxiter=10)
        print(f"Best ARIMA parameters: {model.order} {model.seasonal_order}")
        forecasts, confidence = model.predict(n_periods=24, return_conf_int=True)

        # Append last actual data to train data
        tr_data = pd.concat([tr_data, ts_data], axis=0)

        # Cut train data
        tr_data = tr_data[-2160:]

        # Calculate metrics and add them to a list of metrics
        mae = mean_absolute_error(forecasts, ts_data)
        mse = mean_squared_error(forecasts, ts_data)
        rmse = np.sqrt(mse)

        row = {"MAE": mae, "MSE": mse, "RMSE": rmse}

        # Append values to store them outside the loop
        rows_list.append(row)
        pred_list.append(forecasts)

        counter+=1
    int_end = time.time()

    print(f"Time intermediate for {counter} models:", (int_end - int_start)/60, "min.")


    # Process to store metrics into df
    df_temp = pd.DataFrame(rows_list)
    cols_multiindex = pd.MultiIndex.from_product(
        [[col], ['MAE', 'MSE', 'RMSE']],
          names=['Column name', 'Metrics'])
    df_temp.columns = cols_multiindex
    metrics_df = pd.concat([metrics_df, df_temp], axis=1)

    # Process to store values into df
    df_temp_pred = pd.DataFrame({'True': true_values, 
                                 'Predicted': np.array(pred_list).flatten()}, 
                                 index=true_values.index)
    col_multiindex_pred = pd.MultiIndex.from_product(
        [[col], df_temp_pred.columns], 
        names=['Column name', 'Values'])
    df_temp_pred.columns = col_multiindex_pred
    predictions_df = pd.concat([predictions_df, df_temp_pred], axis=1)
    
end = time.time()

print("Total time:", (end - start)/60, "min.")

In [ ]:
import time 

# Initialize lists to store overall MSE and MAE for each column
overall_mse_dict = {}
overall_mae_dict = {}

for col in train.columns[:-6]:
    print('Column: ', col)
                
    # Initialize lists to store predicted and true values for the whole dataframe
    predicted_values = []
    true_values = []
        
        # Initialize lists to store predicted and true values for this column
        mse_list = []
        mae_list = []

        # Initialize a counter for iterations
        iteration = 0

        # Create a combined_data DataFrame for the current column 
        # Due to the rolling forecast that we make we need to retrain the ARIMA model everytime since ARIMA has no encoder to capture new datapoints
        # Since the ARIMA model only runs on CPU training so many models takes really long
        # Therefore, we always use the last 24 days as training values
        combined_data = train_data[column_name].copy()
        combined_data = combined_data.iloc[-576:] # LAST 576 POINTS??? 576 hours/24 hours =24 days
        
        best_params=()
        
        while iteration <= len(test_data):
            # Calculate the end index of the current rolling window
            end_index = iteration + prediction_length # 0+24

            # Extract the next prediction length steps of test_data for this column
            current_window = test_data[column_name].iloc[iteration:end_index] # FROM 0 TO 24 (NOT INCL)
            # Use PMDARIMA to auto-select the ARIMA model for this column
            model = pm.auto_arima(combined_data, seasonal=False, stepwise=True, trace=False, suppress_warnings=True)

            # Make a forecast for the current window
            forecast, conf_int = model.predict(n_periods=prediction_length, return_conf_int=True)
            
            if len(forecast) != len(current_window):
                forecast = forecast[:len(current_window)]
                
           # Calculate Mean Squared Error (MSE) for the current iteration
            mse = mean_squared_error(current_window, forecast)
            mse_list.append(mse)

            # Calculate Mean Absolute Error (MAE) for the current iteration
            mae = mean_absolute_error(current_window, forecast)
            mae_list.append(mae)

            # Append the current window to combined_data 
            combined_data = pd.concat([combined_data,current_window])
            combined_data.index.freq = 'H'
            # Ensure combined_data only contains the last 24 days to train the next model
            combined_data = combined_data.iloc[-576:]
            
            # Append predicted and true values
            predicted_values.extend(forecast)
            true_values.extend(current_window.values)
            
            # Just some information to know how far we are in the prediction process
            if iteration%768==0:
                print('This is iteration '+str(iteration))
                # Get the best model's parameters
                best_params = model.order
                print(f"Best ARIMA parameters: {best_params}")
            # Increment the iteration counter
            iteration += prediction_length # ON EG 24

        # Calculate the average MSE and MAE for this column and prediction length
        average_mse = np.mean(mse_list)
        average_mae = np.mean(mae_list)
        print(f"Average MSE: {average_mse}, Average MAE: {average_mae}")

        # Store overall MSE and MAE for this column and prediction length in dictionaries
        overall_mse_dict[(prediction_length, column_name)] = average_mse
        overall_mae_dict[(prediction_length, column_name)] = average_mae
        
        # Save predicted and true values as numpy arrays
        predicted_values = np.array(predicted_values)
        true_values = np.array(true_values)
        np.save(os.path.join(results_folder, f'predicted_values_{column_name}__{prediction_length}.npy'), predicted_values)
        np.save(os.path.join(results_folder, f'true_values_{column_name}__{prediction_length}_.npy'), true_values)

# Save overall MSE and MAE to a CSV file
overall_results_df = pd.DataFrame({'Prediction Length': [p[0] for p in overall_mse_dict.keys()],
                                   'Column Name': [p[1] for p in overall_mse_dict.keys()],
                                   'Average Overall MSE': list(overall_mse_dict.values()),
                                   'Average Overall MAE': list(overall_mae_dict.values())})
overall_results_filepath = os.path.join(results_folder, f'overall_results.csv')
overall_results_df.to_csv(overall_results_filepath, index=False)

print("Results saved successfully.")

# Float32

In [ ]:
train, vali, test = split_scale_dataset(time_series_32.iloc[:, :-6], train_size, val_size)

In [ ]:
train.info()

In [ ]:
train_ts = test[-600:-24]
train_ts_exog = test_exog[-600:-24]
test_ts = test[-24:]
test_ts_exog = test_exog[-24:]

In [ ]:
import time 
start = time.time()

model3 = auto_arima(train_ts.iloc[:,1], exogenous=train_exog, stepwise=True, seasonal=True, m=24, maxiter=10)
forecasts, confidence = model3.predict(test_ts.shape[0], exogenous = test_ts_exog, return_conf_int=True)

end = time.time()
print("Total time:", (end - start)/60, "min.")

In [ ]:
plot_predictions(train_ts, test_ts, forecasts)

In [ ]:
df = pd.concat([test_ts.iloc[:, 1], forecasts], axis=1)
df.columns = ['Y_true', 'predicted'] 
df

In [ ]:
np.sqrt(mean_squared_error(forecasts, test_ts.iloc[:, 1]))

# 600 datapoints

# With exogenous

In [ ]:
train, vali, test = split_scale_dataset(time_series.iloc[:, :-6], train_size, val_size)
train_ts = test[-600:-24]
train_ts_exog = test_exog[-600:-24]
test_ts = test[-24:]
test_ts_exog = test_exog[-24:]

In [ ]:
import time 
start = time.time()

model = auto_arima(train_ts.iloc[:,1], exogenous=train_exog, stepwise=True, seasonal=True, m=24, maxiter=10)
forecasts, confidence = model.predict(test_ts.shape[0], exogenous = test_ts_exog, return_conf_int=True)

end = time.time()
print("Total time:", (end - start)/60, "min.")

In [ ]:
model.get_params('order')

In [ ]:
model.order, model.seasonal_order

In [ ]:
def plot_predictions(train, test, forecasts):
    sns.set_style("whitegrid")

    plt.figure(figsize=(10, 6))

    x = np.arange(train.shape[0] + test.shape[0])

    # Combine x and y data into a DataFrame
    data_train = {'Time': x[:train.shape[0]], 'Load': np.array(train.iloc[:, 1])}
    data_forecast = {'Time': x[train.shape[0]:], 'Load': forecasts}

    # Create line plots using Seaborn
    sns.lineplot(data=data_train, x='Time', y='Load', color='blue', label='Train')
    sns.lineplot(data=data_forecast, x='Time', y='Load', color='orange', label='Predicted')

    plt.xlabel("Time (hours)")
    plt.ylabel("Load (MW)")
    plt.legend()

    # Show the plot
    plt.show()

plot_predictions(train_ts, test_ts, forecasts)


In [ ]:
df = pd.concat([test_ts.iloc[:, 1], forecasts], axis=1)
df.columns = ['Y_true', 'predicted'] 
df

In [ ]:
np.sqrt(mean_squared_error(forecasts, test_ts.iloc[:, 1]))

# Without exog

In [ ]:
import time 
start = time.time()

model1 = auto_arima(train_ts.iloc[:,1], stepwise=True, seasonal=True, m=24, maxiter=10)
forecasts1, confidence = model1.predict(test_ts.shape[0], return_conf_int=True)

end = time.time()
print("Total time:", (end - start)/60, "min.")

In [ ]:
plot_predictions(train_ts, test_ts, forecasts1)

In [ ]:
df = pd.concat([test_ts.iloc[:, 1], forecasts1], axis=1)
df.columns = ['Y_true', 'predicted'] 
df

In [ ]:
np.sqrt(mean_squared_error(forecasts1, test_ts.iloc[:, 1]))

In [ ]:
forecasts1

# Seasonal=False

In [ ]:
start = time.time()

model2 = auto_arima(train_ts.iloc[:,1], exogenous = train_ts_exog, stepwise=True, seasonal=False, maxiter=10)
forecasts2, confidence = model2.predict(test_ts.shape[0], return_conf_int=True)

end = time.time()
print("Total time:", (end - start)/60, "min.")

In [ ]:
model2.order, model2.seasonal_order

In [ ]:
plot_predictions(train_ts, test_ts, forecasts2)

In [ ]:
np.sqrt(mean_squared_error(forecasts2, test_ts.iloc[:, 1]))

# 90 days, 2184 (2160)

In [ ]:
train_ts = test[-2184:-24]
train_ts_exog = test_exog[-2184:-24]
test_ts = test[-24:]
test_ts_exog = test_exog[-24:]

In [ ]:
import time 
start = time.time()

model = auto_arima(train_ts.iloc[:,1], exogenous=train_exog, stepwise=True, seasonal=True, m=24, maxiter=10)
forecasts, confidence = model.predict(test_ts.shape[0], exogenous = test_ts_exog, return_conf_int=True)

end = time.time()
print("Total time:", (end - start)/60, "min.")

In [ ]:
model.get_params('order')

In [ ]:
model.order, model.seasonal_order

In [ ]:
plot_predictions(train_ts, test_ts, forecasts)

In [ ]:
df = pd.concat([test_ts.iloc[:, 1], forecasts], axis=1)
df.columns = ['Y_true', 'predicted'] 
df

In [ ]:
np.sqrt(mean_squared_error(forecasts, test_ts.iloc[:, 1]))

# Without exog

In [ ]:
import time 
start = time.time()

model1 = auto_arima(train_ts.iloc[:,1], stepwise=True, seasonal=True, m=24, maxiter=10)
forecasts1, confidence = model1.predict(test_ts.shape[0], return_conf_int=True)

end = time.time()
print("Total time:", (end - start)/60, "min.")

In [ ]:
plot_predictions(train_ts, test_ts, forecasts1)

In [ ]:
df = pd.concat([test_ts.iloc[:, 1], forecasts1], axis=1)
df.columns = ['Y_true', 'predicted'] 
df

In [ ]:
# 0.07556180940940287
np.sqrt(mean_squared_error(forecasts1, test_ts.iloc[:, 1]))

In [ ]:
np.sqrt(mean_squared_error(forecasts1, test_ts.iloc[:, 1]))

In [ ]:
forecasts1

# Seasonal=False

In [ ]:
start = time.time()

model2 = auto_arima(train_ts.iloc[:,1], exogenous = train_ts_exog, stepwise=True, seasonal=False, maxiter=10)
forecasts2, confidence = model2.predict(test_ts.shape[0], return_conf_int=True)

end = time.time()
print("Total time:", (end - start)/60, "min.")

In [ ]:
model2.order, model2.seasonal_order

In [ ]:
plot_predictions(train_ts, test_ts, forecasts2)

In [ ]:
np.sqrt(mean_squared_error(forecasts2, test_ts.iloc[:, 1]))

# With m=24

In [ ]:
# The period for seasonal differencing, m refers to the number of periods in each season.
# For example, m is 4 for quarterly data, 12 for monthly data, or 1 for annual (non-seasonal) data. 
# Default is 1. Note that if m == 1 (i.e., is non-seasonal), seasonal will be set to False. For more 
#information on setting this parameter, see Setting m.


model = auto_arima(train_ts.iloc[:,0], stepwise=True, seasonal=True, m=24, maxiter=10)
# model = pm.auto_arima(combined_data, seasonal=False, stepwise=True, trace=False, suppress_warnings=True)
# Make a forecast for the current window
#forecast, conf_int = model.predict(n_periods=prediction_length, return_conf_int=True)
            

In [ ]:
model.params

In [ ]:
updated_model = model.update(test_ts.iloc[:,0])

# Step 3: Forecast
forecast = updated_model.predict(n_periods=24)


In [ ]:
model2.params

In [ ]:
forecasts, confidence = model.predict(test_ts.shape[0], return_conf_int=True)  # predict N steps into the future

In [ ]:
# Visualize the forecasts (blue=train, green=forecasts)
x = np.arange(train_ts.shape[0] + test_ts.shape[0])
plt.plot(x[:train_ts.shape[0]], np.array(train_ts.iloc[:, 0]), c='blue')
plt.plot(x[train_ts.shape[0]:], forecasts, c='orange')
plt.xlabel("time [h]")
plt.ylabel("load [kW]")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size
plt.plot(forecasts.index, forecasts, label='Predicted', color='blue', marker='o')  # Plot forecasted values
plt.plot(test_ts.index, test_ts.iloc[:, 0], label='True', color='green', marker='s')  # Plot true values
plt.xlabel('Date')  # Set x-axis label
plt.ylabel('Value')  # Set y-axis label
plt.title('Predicted vs. True Values')  # Set plot title
plt.legend()  # Show legend
plt.grid(True)  # Show grid
plt.tight_layout()  # Adjust layout
plt.show()  # Display plot

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
np.sqrt(mean_squared_error(forecasts, test_ts.iloc[:, 0]))

In [ ]:
mean_squared_error(forecasts, test_ts.iloc[:, 0])

In [ ]:
df = pd.concat([test_ts.iloc[:, 0], forecasts], axis=1)
df.columns = ['Y_true', 'predicted'] 
df

# Seasonal=False

In [ ]:
# 2.1 sec??????
model2 = auto_arima(train_ts.iloc[:,0], stepwise=True, seasonal=False, maxiter=3)

forecasts2, confidence2 = model2.predict(test_ts.shape[0], return_conf_int=True)  # predict N steps into the future


In [ ]:
# Visualize the forecasts (blue=train, green=forecasts)
x = np.arange(train_ts.shape[0] + test_ts.shape[0])
plt.plot(x[:train_ts.shape[0]], np.array(train_ts.iloc[:, 0]), c='blue')
plt.plot(x[train_ts.shape[0]:], forecasts2, c='orange')
plt.xlabel("time [h]")
plt.ylabel("load [kW]")
plt.show()